<a href="https://colab.research.google.com/github/Hassanahmed-15/Shakespeare-Plays-Rag-Pipeline-/blob/main/yolo_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("hassan-hpmcy").project("onkar")
version = project.version(2)
dataset = version.download("yolov8-obb")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 117.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to onkar-2 in yolov8-obb:: 100%|██████████| 202/202 [00:00<00:00, 6804.40it/s]


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.4 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model (e.g., YOLOv8n for a smaller model)
model = YOLO('yolov8n.pt')  # You can use 'yolov8s.pt', 'yolov8m.pt', etc.

# Train the model
model.train(
    data='/content/onkar-2/dataset.yaml',      # Path to your dataset config file
    epochs=50,                # Number of training epochs
    imgsz=640,                # Image size (resolution)
    batch=16,                 # Batch size
    name='yolov8_custom_model' # Name for saved results
)

# After training, the model is saved in 'runs/train/yolov8_custom_model'


In [3]:
import yaml

# Define the dataset configuration
dataset_config = {
    'train': '/content/onkar-2/train/images',  # Path to your train images
    'val': '/content/onkar-2/train/images',    # Path to your val images (or same as train if not split)
    'nc': 2,                           # Number of classes
    'names': ['class0', 'class1']       # List of class names
}

# Specify the path to save the YAML file
yaml_file_path = '/content/onkar-2/dataset.yaml'

# Write the configuration to the YAML file
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(dataset_config, yaml_file, default_flow_style=False)

print(f"YAML file created at {yaml_file_path}")


YAML file created at /content/onkar-2/dataset.yaml


In [ ]:
import numpy as np
from ultralytics import YOLO
from pdf2image import convert_from_path
from PIL import Image
from fpdf import FPDF
import os

# Create folder for cropped images
os.makedirs("cropped_notes", exist_ok=True)

# Step 1: Convert PDF pages to images
def pdf_to_images(pdf_path, start_page, end_page):
    pages = convert_from_path(pdf_path, first_page=start_page, last_page=end_page)
    return [np.array(page) for page in pages]

# Step 2: Run YOLOv8 detection
def detect_notes(model, images):
    detections = []
    for img in images:
        results = model(img)
        detected_areas = results[0].to_df().to_pandas()
        detections.append(detected_areas)
    return detections

# Step 3: Crop detected CLASS 0 regions IN ORDER
def crop_detected_regions(img, detections, page_index):
    cropped_image_paths = []

    if detections.empty:
        return cropped_image_paths

    # ✅ FILTER ONLY CLASS 0
    detections_class0 = detections[detections["class"] == 0]

    if detections_class0.empty:
        return cropped_image_paths

    # Extract y1 and x1 for sorting
    detections_class0["y1"] = detections_class0["box"].apply(lambda b: b["y1"])
    detections_class0["x1"] = detections_class0["box"].apply(lambda b: b["x1"])

    # Sort: top → bottom, then left → right
    detections_sorted = detections_class0.sort_values(["y1", "x1"])

    # Zero-padding for correct naming order
    page_str = str(page_index).zfill(4)

    for sorted_idx, (_, row) in enumerate(detections_sorted.iterrows()):
        box = row['box']
        x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']

        cropped_img = img[int(y1):int(y2), int(x1):int(x2)]
        pil_image = Image.fromarray(cropped_img)

        crop_str = str(sorted_idx).zfill(4)

        crop_path = f"cropped_notes/page{page_str}_crop{crop_str}.png"
        pil_image.save(crop_path)

        cropped_image_paths.append(crop_path)

    return cropped_image_paths

# Step 4: Build PDF from class-0 images
def images_to_pdf(image_paths, output_pdf="cropped_notes_output3.pdf"):
    if len(image_paths) == 0:
        print("No class 0 cropped images found.")
        return

    image_paths = sorted(image_paths)

    pdf = FPDF()
    pdf.set_auto_page_break(0)

    for img_path in image_paths:
        pdf.add_page()
        pdf.image(img_path, x=0, y=0, w=210, h=0)

    pdf.output(output_pdf)
    print(f"PDF saved as {output_pdf}")

# Main Function
def process_pdf(pdf_path, start_page, end_page, model):
    images = pdf_to_images(pdf_path, start_page, end_page)
    detections = detect_notes(model, images)

    all_cropped_images = []

    for page_idx, (img, detected_areas) in enumerate(zip(images, detections)):
        cropped_paths = crop_detected_regions(img, detected_areas, page_idx)
        all_cropped_images.extend(cropped_paths)

    images_to_pdf(all_cropped_images, "cropped_notes_output3.pdf")

# Example Usage
if __name__ == "__main__":
    model = YOLO('/content/runs/detect/yolov8_custom_model/weights/best.pt')

    pdf_path = '/content/asulikeitpdf.pdf'

    start_page = 161
    end_page = 304

    process_pdf(pdf_path, start_page, end_page, model)


In [5]:
!pip install pytesseract
!pip install pdf2image
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=7cc067fe2b60f5cf28995492b59f3001938fbfb3300f1f708c369bca7dd00124
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [6]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 0s (465 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121703 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
from google.colab import files
files.download("/content/cropped_notes_output3.pdf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import base64
from pdf2image import convert_from_path
from docx import Document
import openai
import io
import re

# --------------------------------------------------------
# SET YOUR OPENAI API KEY
# --------------------------------------------------------
openai.api_key = ""
# --------------------------------------------------------
# PDF FILE PATH
# --------------------------------------------------------
pdf_path = "/content/cropped_notes_output3.pdf"


# --------------------------------------------------------
# Convert PDF pages → PIL Images
# --------------------------------------------------------
def pdf_to_images(pdf_path):
    print("Converting PDF pages to images...")
    return convert_from_path(pdf_path, dpi=300)


# --------------------------------------------------------
# Encode PIL Image → Base64 String
# --------------------------------------------------------
def encode_image(image):
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")


# --------------------------------------------------------
# CLEAN XML-UNSAFE CHARACTERS
# --------------------------------------------------------
def clean_xml_invalid(text):
    return re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]", "", text)


# --------------------------------------------------------
# GPT-4o Vision: Extract Scholarly Commentary
# --------------------------------------------------------
def gpt_process_image(encoded_image):

    prompt_text = """
You will be given SCHOLARLY COMMENTARY ONLY (no play text).
Read the commentary from the provided image and transcribe it EXACTLY as printed.

DO NOT write “sorry” or ANY extra messages. Output ONLY the transcription.

RULES:
- Do NOT summarize anything.
- Do NOT skip ANY commentary.
- Do NOT merge or restructure commentary.
- Correct OCR/spelling errors only when necessary.
- Reconstruct broken lines cleanly if the scan splits them.
- Keep formatting exactly as in the printed edition.
- Preserve every printed line number exactly as shown.
- Do NOT add headings, labels, or fake line numbers.
- Maintain clean, readable spacing (no random blank gaps).
- Output a clean, continuous, fully corrected transcription.

Return ONLY the scholarly commentary — nothing else.
"""

    response = openai.chat.completions.create(
        model="gpt-4.1",  # MUST be a vision-supported model
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{encoded_image}"
                        }
                    }
                ]
            }
        ]
    )

    # Cleanup invalid XML characters BEFORE returning
    return clean_xml_invalid(response.choices[0].message.content)


# --------------------------------------------------------
# Write final text → DOCX safely
# --------------------------------------------------------
def write_to_docx(full_text, output_path="Scholarly_Commentary.docx"):
    doc = Document()
    safe_text = clean_xml_invalid(full_text)

    for line in safe_text.split("\n"):
        cleaned_line = clean_xml_invalid(line.strip())
        if cleaned_line == "":
            doc.add_paragraph()
        else:
            doc.add_paragraph(cleaned_line)

    doc.save(output_path)
    print(f"\nDONE: Created {output_path}\n")


# --------------------------------------------------------
# MAIN PIPELINE
# --------------------------------------------------------
print("Loading PDF...")
pages = pdf_to_images(pdf_path)

final_output = ""

for idx, page in enumerate(pages):
    print(f"\nProcessing page {idx + 1}/{len(pages)}...")

    encoded_image = encode_image(page)
    gpt_text = gpt_process_image(encoded_image)

    print(f"✓ Page {idx + 1} processed.")

    # Append cleaned text
    final_output += gpt_text + "\n\n"


# Save DOCX
write_to_docx(final_output, "Scholarly_Commentary.docx")


Loading PDF...
Converting PDF pages to images...


In [1]:
!pip install fitz
!pip install python-docx
!pip install PyMuPDF
!pip install tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.9/425.9 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.3 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.1 MB/s eta 0:00:00
